In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression,RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

plt.style.use('ggplot')
%matplotlib inline

In [55]:
df=pd.read_csv('lasso.csv',index_col=0)

In [33]:
df.

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,...,BsmtCond,BsmtExposure,KitchenQual,Functional,FireplaceQu,GarageFinish,PoolQC,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,196.0,856,3,1,8,0,2,548,...,4,1,4,8,1,3,1,0,0,1
2,80.0,9600,0.0,1262,3,1,6,1,2,460,...,4,4,3,8,4,3,1,0,1,0
3,68.0,11250,162.0,920,3,1,6,1,2,608,...,4,2,4,8,4,3,1,1,0,0
4,60.0,9550,0.0,756,3,1,7,1,3,642,...,5,1,4,8,5,2,1,0,0,1
5,84.0,14260,350.0,1145,4,1,9,1,3,836,...,4,3,4,8,4,3,1,0,0,1


In [56]:
SalePrice=df.SalePrice
ridge_df=df.drop('SalePrice',axis=1)
#ridge_df=ridge_df.drop('GarageType_No',axis=1)

### Standardize numeric columns

In [35]:
#check maximum value of each feature to decide which columns we want to normalize
list(map(lambda x:print(x,max(ridge_df[x])),ridge_df))

LotFrontage 313.0
LotArea 215245
MasVnrArea 1600.0
TotalBsmtSF 3206
BedroomAbvGr 8
KitchenAbvGr 3
TotRmsAbvGrd 14
Fireplaces 3
GarageCars 4
GarageArea 1390
WoodDeckSF 857
OpenPorchSF 547
EnclosedPorch 552
ScreenPorch 480
BsmtScore 5.997715851987209
TotalSF 4476
TotalBath 6.0
MSSubClass_30 1
MSSubClass_60 1
MSSubClass_75 1
MSSubClass_80 1
MSSubClass_90 1
MSSubClass_120 1
MSSubClass_160 1
MSSubClass_190 1
MSZoning_FV 1
MSZoning_RH 1
MSZoning_RM 1
Street_Grvl 1
LandContour_Bnk 1
LandContour_HLS 1
LandContour_Low 1
LotConfig_Corner 1
LotConfig_CulDSac 1
LotConfig_FR2 1
Neighborhood_Blmngtn 1
Neighborhood_BrDale 1
Neighborhood_BrkSide 1
Neighborhood_CollgCr 1
Neighborhood_Crawfor 1
Neighborhood_Edwards 1
Neighborhood_Gilbert 1
Neighborhood_Mitchel 1
Neighborhood_NPkVill 1
Neighborhood_NWAmes 1
Neighborhood_NoRidge 1
Neighborhood_NridgHt 1
Neighborhood_OldTown 1
Neighborhood_Sawyer 1
Neighborhood_StoneBr 1
Neighborhood_Veenker 1
BldgType_Duplex 1
BldgType_Twnhs 1
BldgType_TwnhsE 1
HouseStyle

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [57]:
'''The function to standardize the columns seperately. 
df is the dataframe needs the standardization.  
collist is the list of columns which need to be standardized'''


def standize_df(df,collist):
    for col in collist:
        stan_df=df[[col]]
        min_max_scaler = preprocessing.MinMaxScaler()
        standardized=min_max_scaler.fit_transform(stan_df)
        stan2_df=pd.DataFrame(standardized,columns=stan_df.columns,index=stan_df.index)
        df[[col]]=stan2_df
    return df

# Standarize column with maximum values larger than 100 
'''collist=['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','GarageArea','WoodDeckSF','OpenPorchSF',
         'EnclosedPorch','ScreenPorch','TotalSF','YearsAgoBuilt']
'''

#standardize all columns
collist=list(ridge_df.columns) 

ridge_df=standize_df(ridge_df,collist)
ridge_df.head()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,...,BsmtCond,BsmtExposure,KitchenQual,Functional,FireplaceQu,GarageFinish,PoolQC,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,0.150685,0.033420,0.12250,0.266999,0.375,0.333333,0.500000,0.000000,0.50,0.394245,...,0.75,0.000000,0.666667,1.0,0.0,0.666667,0.0,0.0,0.0,1.0
2,0.202055,0.038795,0.00000,0.393637,0.375,0.333333,0.333333,0.333333,0.50,0.330935,...,0.75,1.000000,0.333333,1.0,0.6,0.666667,0.0,0.0,1.0,0.0
3,0.160959,0.046507,0.10125,0.286962,0.375,0.333333,0.333333,0.333333,0.50,0.437410,...,0.75,0.333333,0.666667,1.0,0.6,0.666667,0.0,1.0,0.0,0.0
4,0.133562,0.038561,0.00000,0.235808,0.375,0.333333,0.416667,0.333333,0.75,0.461871,...,1.00,0.000000,0.666667,1.0,0.8,0.333333,0.0,0.0,0.0,1.0
5,0.215753,0.060576,0.21875,0.357143,0.500,0.333333,0.583333,0.333333,0.75,0.601439,...,0.75,0.666667,0.666667,1.0,0.6,0.666667,0.0,0.0,0.0,1.0


In [58]:
# Split the Ames_df into a 80% train-set and a 20% test set.
np.random.seed(19)
testIdxes = np.random.choice(range(1458), size= 292, replace=False)
trainIdxes = list(set(range(1458))-set(testIdxes))
trainX = ridge_df.iloc[trainIdxes]
testX  = ridge_df.iloc[testIdxes]
trainY = SalePrice.iloc[trainIdxes]
testY  = SalePrice.iloc[testIdxes]


#get best lambda
alphaRange = np.linspace(1e-3,200,40) # Use np.linspace to generate 40 alpha
ridge_cv = RidgeCV(alphas = alphaRange,cv=10) #cross validation to get best lambda
ridge_cv.fit(trainX, trainY)
best_alpha=ridge_cv.alpha_
best_alpha=round(best_alpha,3)
best_alpha

0.001

In [59]:

#Train ridge
ridge=Ridge()
ridge.set_params(alpha=best_alpha)
ridge.fit(trainX,trainY)
ridge_intercept = ridge.intercept_
ridge_coef = pd.Series(ridge.coef_,index=ridge_df.columns)
rtrain_score=ridge.score(trainX,trainY)
rtest_score= ridge.score(testX,testY)

In [60]:
print ('intercept \t',ridge_intercept)
print ('train_score(R2)\t',rtrain_score)
print ('test_score(R2) \t',rtest_score)

intercept 	 -22644.65316895346
train_score(R2)	 0.9214628385857856
test_score(R2) 	 0.8778914942770025


In [61]:
ridge_predict = ridge.predict(testX)
RMSE=np.sqrt(mean_squared_error(testY,ridge_predict))
RMSE

23809.706724986947

In [62]:
#Sorted the coefs, the larger the number is , the larger impact it has to the whole model
#showing the top 60 features
sorted_coef=abs(ridge_coef).sort_values(ascending=False)
sorted_coef[0:59]
sorted_coef.BsmtScore

13774.994906815666

In [63]:
PredictionsTest=ridge.predict(testX)
RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
RMSLEvalue

0.15999745546469438

In [54]:
sorted_coef.to_csv('sorted_coef_ridge_new.csv')